In [2]:
!pip install biopython tqdm-joblib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
import torch
from sequence_generator import generate_and_filter_sequences, convert_dna_fasta_to_protein
from models import Generator_lang

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

n_chars = 5                                                                                 
seq_len = 156
batch_size = 64
hidden_g = 192

# Initialize models
generator = Generator_lang(n_chars, seq_len, batch_size, hidden_g).to(device)

saved_model_path = r"/files/private/notebooks/GANs/WUGAN/saved_models2/best_amp/generator_amp_5_epoch_119_score_45.31.pt"

checkpoint = torch.load(saved_model_path, map_location=device)

generator.load_state_dict(checkpoint['model_state_dict'])  # Note: 'model_state_dict' not the whole checkpoint
generator.eval()

sequences, analysis, dna_file = generate_and_filter_sequences(
        generator=generator,
        num_samples=11270-8*640,
        count_atg=False,
        add_atg=False
   )

# Generate sequences and convert to proteins in one go
num_proteins, protein_file = convert_dna_fasta_to_protein(input_fasta=dna_file, add_atg=False)

Using device: cuda

Generation complete!
Found 5294 valid sequences out of 6150 generated
Sequences saved in: /files/private/notebooks/GANs/WUGAN/campr4/valid_sequences.fasta
Analysis saved in: /files/private/notebooks/GANs/WUGAN/campr4/sequence_analysis.txt
Converted 5062 DNA sequences to proteins
Protein sequences saved in: /files/private/notebooks/GANs/WUGAN/campr4/valid_sequences_proteins.fasta


In [9]:
from analyze_amp_probabilities import calculate_averages

# Actual usage example
files = [
    "/files/private/notebooks/GANs/WUGAN/campr4/rf1.txt",
    "/files/private/notebooks/GANs/WUGAN/campr4/svm1.txt",
    "/files/private/notebooks/GANs/WUGAN/campr4/ann1.txt"
]

model_types = [
"Random Forest",
"SVM",
"ANN"
]

calculate_averages(files, model_types)

Percentage above 0.5 (Random Forest): 63.99%
Percentage above 0.8 (Random Forest): 23.40%
Percentage above 0.5 (SVM): 57.51%
Percentage above 0.8 (SVM): 32.18%
Percentage above 0.5 (ANN): 58.95%
Percentage above 0.8 (ANN): 37.47%

Averages across all models:
Average percentage above 0.5: 60.15%
Average percentage above 0.8: 31.02%


In [17]:
from Similarity_fast import calculate_protein_similarity_parallel

protein_file = "/files/private/notebooks/GANs/WUGAN/campr4/valid_sequences_proteins.fasta"

similarity = calculate_protein_similarity_parallel(protein_file)

Analyzing 5089 sequences on all cores...


Alignments:   0%|          | 0/12905740 [15:43<?, ?it/s]
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b2f830ddd30>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Alignments:   0%|          | 0/12946416 [00:00<?, ?it/s]

  0%|          | 0/12946416 [00:00<?, ?it/s]


Overall sequence similarity: 22.65%


In [9]:
from Similarity import calculate_protein_similarity

protein_file = r"C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences_proteins.fasta"

similarity = calculate_protein_similarity(protein_file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kotsgeo\\Documents\\GANs\\Old\\campr4\\valid_sequences_proteins.fasta'

In [2]:
from Similarity import calculate_similarity_between_dna

original_dataset_path = r"C:\Users\kotsgeo\Documents\GANs\all_amp.fasta"
valid_sequences_path = r"C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences.fasta"

similarity = calculate_similarity_between_dna(original_dataset_path, valid_sequences_path)

Comparing 4828 original sequences with 5256 generated sequences...


100%|██████████| 4828/4828 [2:00:17<00:00,  1.49s/it]  


Overall sequence similarity: 48.43%


In [ ]:
# ============= ADD EVALUATION HERE =============

# Import necessary functions
from Bio import SeqIO
import numpy as np
from collections import Counter
from scipy.spatial.distance import jensenshannon
from itertools import product
from amp_evaluator import calculate_peptide_properties, score_peptide

# Verify the file paths
print(f"\nDNA file: {dna_file}")
print(f"Protein file: {protein_file}")

# 1. Calculate JSD Score
print("\n" + "="*60)
print("CALCULATING JSD SCORE")
print("="*60)

# Path to reference sequences
REFERENCE_FASTA = r"C:\Users\kotsgeo\Documents\GANs\Old\AMPdata.txt"  # Your amp.txt file

# Load generated sequences from FASTA
gen_sequences = []
for record in SeqIO.parse(dna_file, "fasta"):
    gen_sequences.append(str(record.seq))

# Load reference sequences from tab-delimited format
ref_sequences = []
try:
    with open(REFERENCE_FASTA, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                # Split by tab to get sequence and label
                parts = line.split('\t')
                if len(parts) >= 1:
                    seq = parts[0].strip().upper()
                    # Verify it's a valid DNA sequence
                    if all(base in 'ATCG' for base in seq):
                        ref_sequences.append(seq)
    
    print(f"Loaded {len(gen_sequences)} generated sequences from {dna_file}")
    print(f"Loaded {len(ref_sequences)} reference sequences from {REFERENCE_FASTA}")
except Exception as e:
    print(f"Error loading reference sequences: {e}")
    ref_sequences = None

# Calculate JSD only if reference sequences are available
if ref_sequences and len(ref_sequences) > 0:
    # Calculate k-mer distributions and JSD
    def get_kmer_distribution_from_strings(seqs, k=6):
        all_kmers = [''.join(p) for p in product('ACGT', repeat=k)]
        kmer_counts = Counter()
        for seq in seqs:
            for i in range(len(seq) - k + 1):
                kmer = seq[i:i+k]
                if all(base in 'ACGT' for base in kmer):
                    kmer_counts[kmer] += 1
        total = sum(kmer_counts.values())
        freq = np.array([kmer_counts[kmer] for kmer in all_kmers], dtype=np.float32)
        return freq / (total + 1e-8)

    k_values = [3, 4, 5, 6]
    jsds = []
    for k in k_values:
        ref_dist = get_kmer_distribution_from_strings(ref_sequences, k=k)
        gen_dist = get_kmer_distribution_from_strings(gen_sequences, k=k)
        jsd_k = jensenshannon(ref_dist, gen_dist)
        jsds.append(jsd_k)
        print(f"JSD for k={k}: {jsd_k:.4f}")

    avg_jsd = np.mean(jsds)
    print(f"\nAverage JSD: {avg_jsd:.4f}")
else:
    avg_jsd = None
    jsds = None
    print("No reference sequences loaded for JSD calculation")

# 2. Calculate AMP Score
print("\n" + "="*60)
print("CALCULATING AMP SCORE")
print("="*60)

# Load protein sequences from FASTA
proteins = []
for record in SeqIO.parse(protein_file, "fasta"):
    proteins.append(str(record.seq))

print(f"Loaded {len(proteins)} protein sequences from {protein_file}")

# Evaluate AMP properties
valid_sequences = 0
passed_all_criteria = 0
criteria_scores = {'length': 0, 'charge': 0, 'hydrophobicity': 0, 'kr_content': 0}

for protein in proteins:
    if len(protein) < 8:
        continue
        
    properties = calculate_peptide_properties(protein)
    if properties is None:
        continue
        
    valid_sequences += 1
    scores = score_peptide(properties)
    
    for criterion, score in scores.items():
        criteria_scores[criterion] += score
        
    if all(score >= 1.0 for score in scores.values()):
        passed_all_criteria += 1

# Calculate final AMP score
if valid_sequences > 0:
    for criterion in criteria_scores:
        criteria_scores[criterion] = (criteria_scores[criterion] / valid_sequences) * 100
    overall_amp_score = np.mean(list(criteria_scores.values()))
else:
    overall_amp_score = 0.0

print(f"\nOverall AMP Score: {overall_amp_score:.2f}%")
print(f"Perfect AMP candidates: {passed_all_criteria}/{len(proteins)} ({(passed_all_criteria/len(proteins)*100 if len(proteins) > 0 else 0):.2f}%)")
print(f"Criteria scores:")
for criterion, score in criteria_scores.items():
    print(f"  {criterion}: {score:.2f}%")

# 3. Save evaluation results
print("\n" + "="*60)
print("SAVING EVALUATION RESULTS")
print("="*60)

import os
evaluation_file = os.path.join(os.path.dirname(dna_file), "evaluation_metrics.txt")

with open(evaluation_file, 'w') as f:
    f.write("SEQUENCE GENERATION EVALUATION METRICS\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"Model: {saved_model_path}\n")
    f.write(f"Generated sequences: {len(sequences)}\n")
    f.write(f"Valid proteins: {num_proteins}\n\n")
    
    if avg_jsd is not None:
        f.write("JSD Analysis:\n")
        f.write(f"Reference file: {REFERENCE_FASTA}\n")
        f.write(f"Average JSD: {avg_jsd:.4f}\n")
        for k, jsd in zip(k_values, jsds):
            f.write(f"  k={k}: {jsd:.4f}\n")
    else:
        f.write("JSD Analysis: Not performed (no reference sequences)\n")
    
    f.write(f"\nAMP Analysis:\n")
    f.write(f"Overall AMP Score: {overall_amp_score:.2f}%\n")
    f.write(f"Perfect AMP candidates: {passed_all_criteria}/{len(proteins)} ({(passed_all_criteria/len(proteins)*100 if len(proteins) > 0 else 0):.2f}%)\n")
    f.write(f"Criteria scores:\n")
    for criterion, score in criteria_scores.items():
        f.write(f"  {criterion}: {score:.2f}%\n")

print(f"Evaluation results saved to: {evaluation_file}")

# Print summary
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"Generated sequences: {len(sequences)}")
print(f"Valid proteins: {num_proteins}")
if avg_jsd is not None:
    print(f"JSD Score: {avg_jsd:.4f}")
else:
    print(f"JSD Score: Not calculated")
print(f"AMP Score: {overall_amp_score:.2f}%")


DNA file: C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences.fasta
Protein file: C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences_proteins.fasta

CALCULATING JSD SCORE
Loaded 5310 generated sequences from C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences.fasta
Loaded 2600 reference sequences from C:\Users\kotsgeo\Documents\GANs\Old\AMPdata.txt
JSD for k=3: 0.0622
JSD for k=4: 0.0893
JSD for k=5: 0.1205
JSD for k=6: 0.1562

Average JSD: 0.1071

CALCULATING AMP SCORE
Loaded 5157 protein sequences from C:\Users\kotsgeo\Documents\GANs\Old\campr4\valid_sequences_proteins.fasta

Overall AMP Score: 86.81%
Perfect AMP candidates: 2370/5157 (45.96%)
Criteria scores:
  length: 100.00%
  charge: 84.66%
  hydrophobicity: 85.29%
  kr_content: 77.30%

SAVING EVALUATION RESULTS
Evaluation results saved to: C:\Users\kotsgeo\Documents\GANs\Old\campr4\evaluation_metrics.txt

FINAL SUMMARY
Generated sequences: 5310
Valid proteins: 5157
JSD Score: 0.1071
AMP Score: 86.81%
